<div align="center">

# Proyecto 3

## Sede central de Cartago

## Escuela de Ingeniería en Computación

## Inteligencia artificial (Gr 02 – IC6200)

### Regresión lineal con descenso del gradiente y selección de características

### Creado por

Esteban Pérez Picado – 2021046572

Joselyn Montero Rodríguez - 2022136356

Manuel A. Rodríguez Murillo - 2021028686

### 24-06-2025

### Semestre I

### Steven Andrey Pacheco Portuguez
</div>

## Introducción
...

## Importación de librerías

In [ ]:
# Python
import os

# Visualización y datos
import numpy as np
import pandas as pd

# Utilidades
from tqdm import tqdm
from joblib import Parallel, delayed


In [1]:
# Conexion con drive
driveBit = 0
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base_path = "/content/drive/MyDrive/archive"
    driveBit = 1
except ImportError:
    #Jupyter local
    #current_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
    #base_path = os.path.join(current_dir, "data")
    print("Colab no detectado, usando ruta local:", base_path)

NameError: name 'base_path' is not defined

In [2]:
# Se verifica que se esté trabajando con la gpu
device = torch.device("cuda" if torch.cuda.is_available () else "cpu")
print ("Is Torch Cuda Available? " + str (torch.cuda.is_available()))

NameError: name 'torch' is not defined